In [1]:
import torch
import os
from PIL import Image
import open_clip

# 模型设置
device = 'mps'  # 或 'cuda' / 'cpu'
model, preprocess, tokenizer = open_clip.create_model_and_transforms(
    'ViT-B-32',
    pretrained='laion2b_s34b_b79k',
    device=device
)

# 图像目录
root = '/Users/lanhaotian/Desktop/python/OpenAI/flask/xhs_school'
lst = os.listdir(root)

# 存储特征和路径
image_features_list = []
image_paths = []

for filename in lst:
    img_path = os.path.join(root, filename)
    try:
        image = Image.open(img_path).convert("RGB")
    except:
        continue  # 跳过无法读取的图像

    image_input = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        features = model.encode_image(image_input)
        features = features / features.norm(dim=-1, keepdim=True)  # 归一化
    image_features_list.append(features.cpu())  # 转到 CPU
    image_paths.append(img_path)

# 合并所有特征为一个大 tensor
database_features = torch.cat(image_features_list)  # shape: [N, D]

# 保存特征和路径
torch.save({
    'features': database_features,
    'paths': image_paths
}, 'image_database.pt')


/Users/lanhaotian/opt/anaconda3/envs/frb/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
